In [1]:
import requests, pytz
import pandas as pd
import numpy as np
from datetime import datetime, time, timedelta
import pandas_market_calendars as mcal
from zoneinfo import ZoneInfo

import pandas_ta as ta



In [2]:
def get_last_full_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = ZoneInfo('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now()
    
    # Ensure current_datetime is timezone-aware
    if current_datetime.tzinfo is None:
        current_datetime = current_datetime.replace(tzinfo=ZoneInfo('UTC'))
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for the current day and the previous day
    schedule = nyse.schedule(start_date=nyse_time.date() - timedelta(days=1), end_date=nyse_time.date())
    
    if not schedule.empty:
        last_close = schedule.iloc[-1]['market_close'].astimezone(nyse_tz)
        
        # If current time is after the last close, that day is the last full trading day
        if nyse_time >= last_close:
            return last_close.date()
        else:
            # Otherwise, we need to find the previous trading day
            previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
            return previous_trading_days[-1].date()
    else:
        # If there's no schedule for today and yesterday (weekend or holiday), 
        # find the last trading day
        previous_trading_days = nyse.valid_days(end_date=nyse_time.date() - timedelta(days=1), start_date=nyse_time.date() - timedelta(days=5))
        return previous_trading_days[-1].date()
    
def get_current_or_next_trading_day(current_datetime=None):
    # Create NYSE calendar
    nyse = mcal.get_calendar('NYSE')
    
    # Get NYSE timezone
    nyse_tz = ZoneInfo('America/New_York')
    
    # Use provided datetime or current time if none provided
    if current_datetime is None:
        current_datetime = datetime.now()
    
    # Ensure current_datetime is timezone-aware
    if current_datetime.tzinfo is None:
        current_datetime = current_datetime.replace(tzinfo=ZoneInfo('UTC'))
    
    # Convert to NYSE time
    nyse_time = current_datetime.astimezone(nyse_tz)
    
    # Get market schedule for today and the next few days
    schedule = nyse.schedule(start_date=nyse_time.date(), end_date=nyse_time.date() + timedelta(days=10))
    
    if not schedule.empty:
        market_open = schedule.iloc[0]['market_open'].astimezone(nyse_tz)
        market_close = schedule.iloc[0]['market_close'].astimezone(nyse_tz)
        
        # If the market is currently open, return today as the current trading day
        if market_open <= nyse_time <= market_close:
            return nyse_time.date()
        else:
            # Otherwise, find the next trading day
            next_trading_days = nyse.valid_days(start_date=nyse_time.date() + timedelta(days=1), end_date=nyse_time.date() + timedelta(days=10))
            return next_trading_days[0].date()
    else:
        # If no schedule for today (market holiday), find the next trading day
        next_trading_days = nyse.valid_days(start_date=nyse_time.date(), end_date=nyse_time.date() + timedelta(days=10))
        return next_trading_days[0].date()

In [3]:
# Get the last full trading day and the current or next trading day
last_trading_day = get_last_full_trading_day()
next_trading_day = get_current_or_next_trading_day()

# Define the SQL query with the specific dates and conditions
query = f"""
WITH LatestEarnings AS (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY act_symbol ORDER BY `date` DESC) AS rn
    FROM `earnings_calendar`
    WHERE `when` IS NOT NULL
)
SELECT *
FROM LatestEarnings
WHERE rn = 1
AND (
    (`date` = '{last_trading_day}' AND `when` = 'After market close') OR
    (`date` = '{next_trading_day}' AND `when` = 'Before market open')
)
ORDER BY `act_symbol` ASC;
"""

# URL encode the query
encoded_query = requests.utils.quote(query)

# Set the DoltHub API endpoint and parameters
endpoint = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/earnings/master?q={encoded_query}"


In [10]:
# Make the request to DoltHub
response = requests.get(endpoint)
response.raise_for_status()  # Check for errors

# Check the content of the response
data = response.json()

# Handle the response and convert to a pandas DataFrame if successful
if data.get('query_execution_status') == 'Success':  # Updated condition to 'Success'
    df = pd.DataFrame(data['rows'])
    df = df.rename(columns={'act_symbol': 'symbol'})
    df = df[['symbol', 'date', 'when']]
    print(df)
    earnings_symbols = df.symbol.tolist()
else:
    print(f"Query Error: {data.get('query_execution_message')}")

  symbol        date                when
0    AYI  2024-10-01  Before market open
1    MKC  2024-10-01  Before market open
2  MKC.V  2024-10-01  Before market open
3   PAYX  2024-10-01  Before market open
4   POCI  2024-09-30  After market close
5   SUUN  2024-09-30  After market close
6   TRAK  2024-09-30  After market close
7   UNFI  2024-10-01  Before market open
8   VRAR  2024-09-30  After market close


In [28]:
# Define the SQL query to find the maximum date
query_max_date = """
SELECT MAX(`date`) AS max_date
FROM `volatility_history`
"""

# URL encode the query
encoded_query_max_date = requests.utils.quote(query_max_date)

# Set the DoltHub API endpoint and parameters for max date
endpoint_max_date = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_max_date}"

# Make the request to DoltHub for the max date
response_max_date = requests.get(endpoint_max_date)
response_max_date.raise_for_status()  # Check for errors

# Convert the response to a pandas DataFrame
data_max_date = response_max_date.json()

# Extract the maximum date from the response
if data_max_date.get('query_execution_status') == 'Success' and data_max_date.get('rows'):
    max_date = data_max_date['rows'][0]['max_date']
    print("Looking up vol data for:", max_date)
else:
    print(f"Query Error: {data_max_date.get('query_execution_message')}")
    max_date = None

Looking up vol data for: 2024-09-27


In [31]:
new_date = get_last_full_trading_day(datetime.strptime(max_date, '%Y-%m-%d').replace(tzinfo=ZoneInfo('UTC')))
new_date = new_date.strftime('%Y-%m-%d')
print(new_date)

2024-09-26


In [34]:
if max_date and earnings_symbols:
    # Convert earnings_symbols list to a comma-separated string of quoted symbols
    symbols_str = ', '.join(f"'{symbol}'" for symbol in earnings_symbols)
    
    # Define the SQL query to get volatility history for the max date and earnings symbols
    query_volatility = f"""
    SELECT *
    FROM `volatility_history`
    WHERE `date` = '{max_date}'
    AND `act_symbol` IN ({symbols_str})
    ORDER BY `act_symbol` ASC;
    """
    # AND `act_symbol` = {df.act_symbol}

    # URL encode the query
    encoded_query_volatility = requests.utils.quote(query_volatility)

    # Set the DoltHub API endpoint and parameters for volatility history
    endpoint_volatility = f"https://www.dolthub.com/api/v1alpha1/post-no-preference/options/master?q={encoded_query_volatility}"

    # Print the endpoint to verify
    # print("Volatility History Endpoint:", endpoint_volatility)

    # Make the request to DoltHub for volatility history
    response_volatility = requests.get(endpoint_volatility)
    response_volatility.raise_for_status()  # Check for errors

    # Convert the response to a pandas DataFrame
    data_volatility = response_volatility.json()

    # Debug print the response data
    print("Response JSON for Volatility History:", data_volatility)

    # Check and convert the response to a pandas DataFrame
    if data_volatility.get('query_execution_status') == 'RowLimit':
        if data_volatility.get('rows'):
            df_volatility = pd.DataFrame(data_volatility['rows'])
            # Display the DataFrame
            print(df_volatility)
        else:
            print("No data found for the maximum date.")
    else:
        df_volatility = pd.DataFrame(data_volatility['rows'])
else:
    print("No valid max date found.")

Response JSON for Volatility History: {'query_execution_status': 'Success', 'query_execution_message': '', 'repository_owner': 'post-no-preference', 'repository_name': 'options', 'commit_ref': 'master', 'sql_query': "\n    SELECT *\n    FROM `volatility_history`\n    WHERE `date` = '2024-09-27'\n    AND `act_symbol` IN ('AYI', 'MKC', 'MKC.V', 'PAYX', 'POCI', 'SUUN', 'TRAK', 'UNFI', 'VRAR')\n    ORDER BY `act_symbol` ASC;\n    ", 'schema': [{'columnName': 'date', 'columnType': 'date'}, {'columnName': 'act_symbol', 'columnType': 'varchar(64)'}, {'columnName': 'hv_current', 'columnType': 'decimal(5,4)'}, {'columnName': 'hv_week_ago', 'columnType': 'decimal(5,4)'}, {'columnName': 'hv_month_ago', 'columnType': 'decimal(5,4)'}, {'columnName': 'hv_year_high', 'columnType': 'decimal(5,4)'}, {'columnName': 'hv_year_high_date', 'columnType': 'date'}, {'columnName': 'hv_year_low', 'columnType': 'decimal(5,4)'}, {'columnName': 'hv_year_low_date', 'columnType': 'date'}, {'columnName': 'iv_current',

In [33]:
df_volatility

,date,act_symbol,hv_current,hv_week_ago,hv_month_ago,hv_year_high,hv_year_high_date,hv_year_low,hv_year_low_date,iv_current,iv_week_ago,iv_month_ago,iv_year_high,iv_year_high_date,iv_year_low,iv_year_low_date
0,2024-09-26,AYI,0.2346,0.2470,None,0.4079,2024-08-23,0.1537,2024-03-28,0.4417,0.3592,None,0.4417,2024-09-26,0.1855,2024-05-22
1,2024-09-26,MKC,0.1257,0.1360,None,0.3780,2023-11-03,0.1257,2024-09-26,0.2650,0.2865,None,0.3486,2024-03-25,0.1804,2024-05-10
2,2024-09-26,PAYX,0.1407,0.1433,None,0.2676,2024-08-07,0.1145,2024-03-20,0.2821,0.2710,None,0.3273,2024-08-05,0.1578,2024-05-20
3,2024-09-26,UNFI,0.4526,0.4510,None,1.0867,2023-11-06,0.3005,2024-01-31,0.7008,0.6961,None,0.9310,2023-12-05,0.4089,2024-03-27


In [21]:
df_vol = df_volatility[['act_symbol', 'date', 'hv_current', 'iv_current']].astype({'hv_current': 'float','iv_current': 'float'})
df_vol['vol_premium'] = df_vol['iv_current']/df_vol['hv_current']
df_vol.sort_values(by='vol_premium', ascending=False)
# df_vol['vol_premium_rank_pct'] = df_vol['vol_premium_rank']/df_vol['vol_premium_rank'].max()
# df_vol['vol_premium_rank_pct_diff'] = df_vol['vol_premium_rank_pct'].diff()
# df_vol['vol_premium_rank_pct_diff_shift'] = df_vol['vol_premium_rank_pct_diff'].shift(-1)
# df_vol['vol_premium_rank_pct_diff_shift_rank'] = df_vol['vol_premium_rank_pct_diff_shift'].rank(ascending=False)
# df_vol['vol_premium_rank_pct_diff_shift_rank_pct'] = df_vol['vol_premium_rank_pct_diff_shift_rank']/df_vol['vol_premium_rank_pct_diff_shift_rank'].max()
# df_vol['vol_premium_rank_pct_diff_shift_rank_pct_diff'] = df_vol['vol_premium_rank_pct_diff_shift_rank_pct'].diff()
# df_vol['vol_premium_rank_pct_diff_shift_rank_pct_diff_shift'] = df_vol['vol_premium_rank_pct_diff_shift_rank_pct_diff'].shift(-1)
# df_vol['vol_premium_rank_pct_diff_shift_rank_pct_diff_shift_rank'] = df_vol['vol_premium_rank_pct_diff_shift_rank_pct_diff_shift'].rank(ascending=False)

,act_symbol,date,hv_current,iv_current,vol_premium
1,MKC,2024-09-27,0.1156,0.2667,2.307093
2,PAYX,2024-09-27,0.1403,0.3107,2.214540
0,AYI,2024-09-27,0.2300,0.4497,1.955217
3,UNFI,2024-09-27,0.4331,0.7526,1.737705


In [14]:
df_volatility[df_volatility.act_symbol == 'CODA']

,date,act_symbol,hv_current,hv_week_ago,hv_month_ago,hv_year_high,hv_year_high_date,hv_year_low,hv_year_low_date,iv_current,iv_week_ago,iv_month_ago,iv_year_high,iv_year_high_date,iv_year_low,iv_year_low_date


In [21]:
from ib_async import *



util.startLoop()  # Needed in script mode
ib = IB()

ib.connect('127.0.0.1', 7497, clientId=5)

<IB connected to 127.0.0.1:7497 clientId=5>

In [36]:
stock = Stock('MSFT', 'SMART', 'USD')
ib.qualifyContracts(stock)

[Stock(conId=272093, symbol='MSFT', exchange='SMART', primaryExchange='ISLAND', currency='USD', localSymbol='MSFT', tradingClass='NMS')]

In [42]:
chains = await ib.reqSecDefOptParamsAsync(stock.symbol, '', stock.secType, stock.conId)

In [43]:
util.df(chains)

,exchange,underlyingConId,tradingClass,multiplier,expirations,strikes
0,AMEX,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
1,BOX,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
2,CBOE2,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
3,BATS,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
4,EMERALD,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
5,MEMX,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
6,PHLX,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
7,SAPPHIRE,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
8,NASDAQOM,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."
9,PSE,272093,MSFT,100,"[20240913, 20240920, 20240927, 20241004, 20241...","[110.0, 115.0, 120.0, 125.0, 130.0, 135.0, 140..."


In [24]:
async def get_option_chain(symbol):

    stock = Stock(symbol, 'SMART', 'USD')
    await ib.qualifyContractsAsync(stock)

    chains = await ib.reqSecDefOptParamsAsync(
        stock.symbol, '', stock.secType, stock.conId)

    if not chains:
        print(f"No option chain found for {symbol}")
        await ib.disconnectAsync()
        return None

    chain = next(c for c in chains if c.exchange == 'SMART')
    
    contracts = [
        ib.Option(symbol, chain.expirations[0], strike, 'C', 'SMART', tradingClass=chain.tradingClass)
        for strike in chain.strikes
    ]

    contracts = await ib.qualifyContractsAsync(*contracts)
    
    tickers = await ib.reqTickersAsync(*contracts)
    
    await ib.disconnectAsync()
    
    return tickers, stock

async def calculate_avg_iv(symbol):
    tickers, stock = await get_option_chain(symbol)
    
    if not tickers:
        return None
    
    stock_price = stock.marketPrice()
    
    # Sort tickers by strike price
    sorted_tickers = sorted(tickers, key=lambda x: x.contract.strike)
    
    # Find the index of the at-the-money option
    atm_index = next(i for i, t in enumerate(sorted_tickers) if t.contract.strike > stock_price)
    
    # Get 5 strikes above and below
    relevant_tickers = sorted_tickers[max(0, atm_index-5):min(len(sorted_tickers), atm_index+6)]
    
    # Calculate average IV
    total_iv = sum(t.optPrice for t in relevant_tickers if t.optPrice is not None)
    avg_iv = total_iv / len(relevant_tickers)
    
    return avg_iv

# Run the async function
import asyncio

symbol = 'CODA'
avg_iv = asyncio.get_event_loop().run_until_complete(calculate_avg_iv(symbol))

if avg_iv is not None:
    print(f"Average Implied Volatility for {symbol}: {avg_iv:.2f}")
else:
    print(f"Unable to calculate average IV for {symbol}")


No option chain found for CODA


AttributeError: 'IB' object has no attribute 'disconnectAsync'